In [ ]:
import psutil

def show_memory_usage():
    """Displays system memory usage in GB and percentage."""
    memory = psutil.virtual_memory()
    print(f"Total memory: {memory.total / (1024 ** 3):.2f} GB")
    print(f"Used memory: {memory.used / (1024 ** 3):.2f} GB")
    print(f"Available memory: {memory.available / (1024 ** 3):.2f} GB")
    print(f"Percentage used: {memory.percent}%")

# Execute memory check
show_memory_usage()


# First Training

In [ ]:
import pandas as pd

# 📁 Load the initial training batch
first_batch_path = "../data/First batch.xlsx"
union_df = pd.read_excel(first_batch_path)

# 🔍 Initial overview
print("✅ First batch loaded successfully")
print("Dataset shape:", union_df.shape)
print("Column names:", union_df.columns.tolist())
print("\nMissing values per column:\n", union_df.isnull().sum())

# Display first rows
display(union_df.head())

## Exploration Data Analysis

### Distributions, Outliers and Exploration

In [ ]:
from ydata_profiling import ProfileReport

zone = 4
# Filtering by zone
df_zone = union_df[union_df["Soot Level Zone in the DPF"] == zone]

# EDA Report
profile = ProfileReport(df_zone, title=f"EDA Report - Zone {zone}", explorative=True)

# EDA Report display
profile.to_notebook_iframe()

### Heatmap

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 🔁 Iterate over soot level zones
for zone in [3.0, 4.0]:

    # Subset data by zone
    df_zone = union_df[union_df["Soot Level Zone in the DPF"] == zone]
    print(f"📊 Generating heatmap for Zone {int(zone)} | Shape: {df_zone.shape}")

    # 🔢 Rename columns with numeric labels
    original_columns = df_zone.columns.to_list()
    column_mapping = {col: str(i) for i, col in enumerate(original_columns)}
    df_zone_renamed = df_zone.rename(columns=column_mapping)

    # 🧽 Drop the zone column (not needed in correlation matrix)
    df_zone_renamed = df_zone_renamed.drop(columns=column_mapping["Soot Level Zone in the DPF"])

    # 📈 Compute correlation matrix
    corr_matrix = df_zone_renamed.corr()

    # 🎨 Plotting parameters
    fig_size = (14, 10)
    font_size_title = 24
    font_size_labels = 18
    font_size_annot = 18
    font_size_colorbar = 18

    # 🔥 Generate correlation heatmap
    plt.figure(figsize=fig_size)
    sns.heatmap(
        corr_matrix,
        annot=True,
        cmap="coolwarm",
        fmt=".2f",
        linewidths=0.5,
        vmin=-1,
        vmax=1,
        annot_kws={"size": font_size_annot},
        cbar_kws={"shrink": 0.8}
    )
    plt.title(f"Correlation Heatmap - Zone {int(zone)}", fontsize=font_size_title)
    plt.xticks(fontsize=font_size_labels, rotation=45, ha="right")
    plt.yticks(fontsize=font_size_labels)
    plt.gca().collections[0].colorbar.ax.tick_params(labelsize=font_size_colorbar)
    plt.tight_layout()
    plt.show()

    # 📘 Display column name mapping table
    legend_df = pd.DataFrame({
        "Original Name": original_columns,
        "Variable Code": [column_mapping[col] for col in original_columns]
    })
    display(legend_df)


### Scatters

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 🎯 Define target variable
target = "Delta Time [h]"
zone = 3

# 📊 Filter dataset by zone and drop missing values
plot_df = union_df[union_df["Soot Level Zone in the DPF"] == zone].dropna()

# 📌 Selected variables to compare against the target
selected_cols = [
    "Simulated soot content in the particulate filter [kg]",
    "fuel_intensity_index_prev_zone",
    "fuel_intensity_index"    
]

font_size = 16

# 🔁 Plot each variable vs. the target
for col in selected_cols:
    if col == target:
        continue  # Skip if column is the same as the target
    
    plt.figure(figsize=(6, 4))
    sns.scatterplot(
        data=plot_df,
        x=col,
        y=target,
        s=30, 
        color="#2C3E50",
        edgecolor="black"
    )
    plt.xlabel(col, fontsize=font_size)
    plt.ylabel(target, fontsize=font_size)
    plt.title(f"Engineered Variable vs Target – Zone {zone}", fontsize=font_size + 2)
    plt.tight_layout()
    plt.yticks(fontsize=font_size)
    plt.xticks(fontsize=font_size)
    plt.grid(True)
    plt.show()


# Models

## Zone 3

### Training and Test

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import smogn

# 🔘 Toggle SMOGN application
smogn_activation = False

# 🎯 Define target variable
target = "Delta Time [h]"

# 🧼 Filter Zone 3 data
zone_df = union_df[union_df["Soot Level Zone in the DPF"] == 3.0].copy()

# ✅ Selected features (excluding categorical zone)
feature_cols = [
    "fuel_intensity_index",
    "Simulated Soot Content in the Particulate Filter [kg]"
]

# 📊 Define custom bin edges (adjust as needed)
bin_edges = [0, 0.015, 0.020, 0.025, 0.030, 1.0]

# 🔀 Create stratification bins
zone_df["soot_bin"] = pd.cut(
    zone_df["fuel_intensity_index"],
    bins=bin_edges,
    include_lowest=True
)

# 🎯 Split data stratified by soot_bin
X = zone_df[feature_cols]
y = zone_df[target]
stratify_col = zone_df["soot_bin"]

X_train, X_test, y_train, y_test, strat_train, strat_test = train_test_split(
    X, y, stratify_col, test_size=0.2, random_state=42, stratify=stratify_col
)

# 📦 Prepare training DataFrame
train_df = X_train.copy()
train_df[target] = y_train

# 🧪 Define SMOGN function
def apply_smogn(df, target_var):
    """
    Applies SMOGN oversampling to rebalance the training set based on the target.
    Returns the new DataFrame.
    """
    print("⚙️ Applying SMOGN oversampling...")
    smogn_df = smogn.smoter(
        data=df,
        y=target_var,
        k=7,
        samp_method="balance",
        rel_thres=0.9,
        rel_method="auto",
        rel_xtrm_type="high",
        rel_coef=0.8
    )
    return smogn_df

# 🧪 Apply SMOGN if activated
if smogn_activation:
    train_df = apply_smogn(train_df, target)

# 🔄 Redefine training sets using the (possibly) updated DataFrame
X_train = train_df[feature_cols]
y_train = train_df[target]

# ⚖️ Scale features
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🤖 Define models and hyperparameter grids
models_grids = {
    "Linear Regression": (LinearRegression(), {}),
    "Ridge Regression": (
        Ridge(),
        {"alpha": [0.01, 0.1, 1, 10, 50, 100]}
    ),
    "Decision Tree": (
        DecisionTreeRegressor(random_state=42),
        {"max_depth": [2, 4, 6, 10, None], "min_samples_split": [2, 5, 10]}
    ),
    "KNN": (
        KNeighborsRegressor(),
        {"n_neighbors": [3, 5, 7, 9, 11], "weights": ["uniform", "distance"]}
    ),
}

# 📊 Train and evaluate
results = []

for name, (model, params) in models_grids.items():
    print(f"\n🔧 Model: {name}")
    grid = GridSearchCV(model, params, cv=LeaveOneOut(), scoring="r2", n_jobs=-1)
    grid.fit(X_train_scaled, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test_scaled)

    # Metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Model": name,
        "R2": round(r2, 4),
        "MAE": round(mae, 4),
        "RMSE": round(rmse, 4),
        "Best Parameters": grid.best_params_
    })

    font_size = 14

    # 📈 Actual vs Predicted plot
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_test, y=y_pred, s=30, color="#2C3E50", edgecolor="black")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label="Ideal")
    plt.xlabel("Actual ΔTime [h]", fontsize=font_size)
    plt.ylabel("Predicted ΔTime [h]", fontsize=font_size)
    plt.title(f"{name} - Actual vs Predicted (Zone 3)", fontsize=font_size + 2)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

    # 📉 Residuals plot
    residuals = y_test - y_pred
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_pred, y=residuals, s=30, color="#34495E", edgecolor="black")
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Predicted ΔTime [h]", fontsize=font_size)
    plt.ylabel("Residuals [h]", fontsize=font_size)
    plt.title(f"{name} - Residuals vs Predicted (Zone 3)", fontsize=font_size + 2)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

# 📋 Final results table
results_df = pd.DataFrame(results)
display(results_df)


### Explainability

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns

# 📌 Parameters
target = "Delta Time [h]"
feature_cols = [
    "fuel_intensity_index",
    "Simulated Soot Content in the Particulate Filter [kg]"
]
zone = 3

# 📦 Filter dataset by DPF soot zone and create stratification bins
zone_df = union_df[union_df["Soot Level Zone in the DPF"] == zone].copy()
bin_edges = [0, 0.015, 0.020, 0.025, 0.030, 1.0]
zone_df["soot_bin"] = pd.cut(zone_df["Fuel Intensity Index"], bins=bin_edges, include_lowest=True)

# 🧪 Split data
X = zone_df[feature_cols]
y = zone_df[target]
stratify_col = zone_df["soot_bin"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=stratify_col, test_size=0.2, random_state=42
)

# 🔧 Scale features
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🧠 Train KNN model
knn = KNeighborsRegressor(n_neighbors=3, weights='uniform')
knn.fit(X_train_scaled, y_train)

# 📊 Compute permutation importance
result = permutation_importance(
    knn, X_test_scaled, y_test,
    n_repeats=1000, random_state=42, scoring="r2"
)

# 🏷️ Rename features for visualization
pretty_names = {
    "fuel_intensity_index": "Fuel Intensity Index",
    "Simulated Soot Content in the Particulate Filter [kg]": "Simulated Soot Content\nin the Particulate Filter"
}

# 📋 Build importance DataFrame
importance_df = pd.DataFrame({
    "Variable": [pretty_names[col] for col in feature_cols],
    "Mean Importance": result.importances_mean,
    "Std Dev": result.importances_std
}).sort_values(by="Mean Importance", ascending=False)

# 🎨 Plot results
plt.figure(figsize=(8, 5))
sns.barplot(
    data=importance_df,
    x="Mean Importance",
    y="Variable",
    palette="Blues_d",
    edgecolor="black"
)
plt.title(f"Permutation Importance – Zone {zone}", fontsize=24)
plt.xlabel("Mean Decrease in R²", fontsize=18)
plt.ylabel("")  # Hide axis label
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

# ✅ Display importance table
display(importance_df)


### Final model for preduction test

In [ ]:
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import os, joblib
import numpy as np

# 🎯 Target variable
target = "Delta_time [h]"

# 🧼 Filter Zone 3 and remove outliers (same as previous step)
zone_df = union_df[union_df["Soot Level Zone in the DPF"] == 3.0].copy()
np.random.seed(42)

# ✅ Predictor variables
feature_cols = [
    "fuel_intensity_index",
    "Simulated soot content in the particulate filter [kg]"
]

X_final = zone_df[feature_cols]
y_final = zone_df[target]

# 🔄 Scale features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_final)

# ✅ Leave-One-Out Cross-Validation
model = LinearRegression()
scores = cross_val_score(model, X_scaled, y_final, cv=LeaveOneOut(), scoring="r2")

# ⚙️ Train final model
model.fit(X_scaled, y_final)

# 💾 Save model and scaler
output_path = "../models/"

joblib.dump(model, os.path.join(output_path, "regression_model_zone_3.pkl"))
joblib.dump(scaler, os.path.join(output_path, "feature_scaler_zone_3.pkl"))

print("✅ Model and scaler saved successfully.")


## Zone 4

### Training and Test

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import smogn

# 🎯 Target and zone
target = "Delta Time [h]"
zone = 4  
smogn_enabled = True  # Toggle SMOGN application

# 📦 Filter data by soot zone
zone_df = union_df[union_df["Soot Level Zone in the DPF"] == float(zone)].copy()

# ✅ Feature columns
feature_cols = [
    "fuel_intensity_index_prev_zone",
    "Simulated Soot Content in the Particulate Filter [kg]"
]

# 🧩 Create bins for stratification
bin_edges = [0, 0.05, 0.10, 0.15, 1.0]
zone_df["soot_bin"] = pd.cut(
    zone_df["fuel_intensity_index_prev_zone"],
    bins=bin_edges,
    include_lowest=True
)

# 🧪 Train-test split
X = zone_df[feature_cols]
y = zone_df[target]
stratify_col = zone_df["soot_bin"]

X_train, X_test, y_train, y_test, strat_train, strat_test = train_test_split(
    X, y, stratify_col, test_size=0.2, random_state=42, stratify=stratify_col
)

# 🧬 Rebuild train DataFrame
train_df = X_train.copy()
train_df[target] = y_train

# 🔁 Optional SMOGN function
def apply_smogn_if_enabled(df, target_column, enabled=True):
    if not enabled:
        print("✅ SMOGN not applied.")
        return df.copy()
    
    print("⚠️ Applying SMOGN...")
    df_smogn = smogn.smoter(
        data=df,
        y=target_column,
        k=7,
        samp_method="balance",
        rel_thres=0.9,
        rel_method="auto",
        rel_xtrm_type="high",
        rel_coef=0.8
    )
    print("✅ SMOGN applied.")
    return df_smogn

# 🧪 Apply SMOGN if enabled
train_df = apply_smogn_if_enabled(train_df, target, smogn_enabled)

# 🔄 Redefine train sets
X_train = train_df[feature_cols]
y_train = train_df[target]

# 🔧 Scale inputs
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🤖 Models and hyperparameters
models_grids = {
    "Linear Regression": (LinearRegression(), {}),
    
    "Ridge Regression": (
        Ridge(), 
        {"alpha": [0.01, 0.1, 1, 10, 50, 100]}
    ),
    
    "Decision Tree": (
        DecisionTreeRegressor(random_state=42), 
        {
            "max_depth": [2, 4, 6, 10, None],
            "min_samples_split": [2, 5, 10]
        }
    ),
    
    "KNN": (
        KNeighborsRegressor(), 
        {
            "n_neighbors": [3, 5, 7, 9, 11],
            "weights": ["uniform", "distance"]
        }
    )
    # Optional: Add other models if needed
}

# 📊 Train and evaluate
results = []

for name, (model, params) in models_grids.items():
    print(f"\n🔧 Training model: {name}")
    grid = GridSearchCV(model, params, cv=LeaveOneOut(), scoring="r2", n_jobs=-1)
    grid.fit(X_train_scaled, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test_scaled)

    # Evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    results.append({
        "Model": name,
        "R2": round(r2, 4),
        "MAE": round(mae, 4),
        "RMSE": round(rmse, 4),
        "Best Parameters": grid.best_params_
    })

    font_size = 14

    # 📈 Actual vs Predicted
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_test, y=y_pred, s=30, color="#2C3E50", edgecolor="black")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel("Actual ΔTime [h]", fontsize=font_size)
    plt.ylabel("Predicted ΔTime [h]", fontsize=font_size)
    plt.title(f"{name} - Actual vs Predicted (Zone {zone})", fontsize=font_size + 2)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

    # 📉 Residuals
    residuals = y_test - y_pred
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=y_pred, y=residuals, s=30, color="#34495E", edgecolor="black")
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel("Predicted ΔTime [h]", fontsize=font_size)
    plt.ylabel("Residuals [h]", fontsize=font_size)
    plt.title(f"{name} - Residuals vs Predicted (Zone {zone})", fontsize=font_size + 2)
    plt.tight_layout()
    plt.grid(True)
    plt.show()

# 🧾 Final results table
results_df = pd.DataFrame(results)
display(results_df)


### Explainability

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns

# 📌 Parameters
target = "Delta_time [h]"
zone = 4  

# ✅ Feature columns
feature_cols = [
    "fuel_intensity_index_prev_zone",
    "Simulated soot content in the particulate filter [kg]"
]

# 📦 Filter data by zone and create stratification bins
zone_df = union_df[union_df["Soot Level Zone in the DPF"] == zone].copy()
bin_edges = [0, 0.05, 0.10, 0.15, 1]  
zone_df["soot_bin"] = pd.cut(zone_df["fuel_intensity_index_prev_zone"], bins=bin_edges, include_lowest=True)

# 🧪 Train-test split
X = zone_df[feature_cols]
y = zone_df[target]
stratify_col = zone_df["soot_bin"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=stratify_col, test_size=0.2, random_state=42
)

# 🔧 Scaling
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🧠 Best model from tuning
knn = KNeighborsRegressor(n_neighbors=3, weights='uniform')
knn.fit(X_train_scaled, y_train)

# 📊 Permutation Importance
result = permutation_importance(
    knn, X_test_scaled, y_test, 
    n_repeats=1000, random_state=42, scoring="r2"
)

# 📋 Feature name mapping for plot
pretty_names = {
    "fuel_intensity_index_prev_zone": "Fuel intensity index\n(previous zone)",
    "Simulated soot content in the particulate filter [kg]": "Simulated soot content\nin DPF"
}

# 📄 Create DataFrame with pretty names
importance_df = pd.DataFrame({
    "Variable": [pretty_names[col] for col in feature_cols],
    "Mean Importance": result.importances_mean,
    "Std Dev": result.importances_std
}).sort_values(by="Mean Importance", ascending=False)

# 🎨 Plotting
plt.figure(figsize=(8, 5))
sns.barplot(
    data=importance_df,
    x="Mean Importance",
    y="Variable",
    palette="Blues_d",
    edgecolor="black"
)
plt.title(f"Permutation Importance – Zone {zone}", fontsize=24)
plt.xlabel("Mean Decrease in R²", fontsize=18)
plt.ylabel("")
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

# ✅ Show table (for Jupyter)
display(importance_df)


### Final model for preduction test

In [ ]:
import os
import joblib
import numpy as np
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import KNeighborsRegressor

# 🎯 Target
target = "Delta_time [h]"

# 🧼 Filter zone 4
zone_df = union_df[union_df["Soot Level Zone in the DPF"] == 4.0].copy()

# ✅ Predictor variables
feature_cols = [
    "fuel_intensity_index_prev_zone",
    "Simulated soot content in the particulate filter [kg]"
]

X_final = zone_df[feature_cols]
y_final = zone_df[target]

# 🔄 Scaling
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_final)

# 🧠 Model with validated hyperparameters
model = KNeighborsRegressor(n_neighbors=3, weights="uniform")

# 📊 Leave-One-Out Cross-Validation
cv = LeaveOneOut()
scores = cross_val_score(model, X_scaled, y_final, cv=cv, scoring="r2", n_jobs=-1)

# ⚙️ Final training on all data
final_model = model.fit(X_scaled, y_final)

# 💾 Save model and scaler (generic path and filenames for GitHub)
output_path = "./models"
os.makedirs(output_path, exist_ok=True)

joblib.dump(final_model, os.path.join(output_path, "knn_model_zone4.pkl"))
joblib.dump(scaler, os.path.join(output_path, "scaler_knn_zone4.pkl"))

print("✅ KNN model for zone 4 and scaler saved successfully.")


# Production Test

## Loading

In [ ]:
import pandas as pd

# Load the test batch
second_batch_path = "../data/Second batch.xlsx"
final_df_preparation_new = pd.read_excel(second_batch_path)

# Initial overview
print("✅ Second batch loaded successfully")
print("Dataset shape:", final_df_preparation_new.shape)
print("Column names:", final_df_preparation_new.columns.tolist())
print("\nMissing values per column:\n", final_df_preparation_new.isnull().sum())

# Display first rows
display(final_df_preparation_new.head())

## Exploration Data Analysis

### Distributions, Outliers and Exploration

In [ ]:
from ydata_profiling import ProfileReport

zone = 4
# Filtrar por zona
df_zone = final_df_preparation_new[final_df_preparation_new["Soot Level Zone in the DPF"] == zone]

# Crear el informe del EDA
profile = ProfileReport(df_zone, title=f"EDA Report - Zone {zone}", explorative=True)

# Mostrar el reporte en el notebook
profile.to_notebook_iframe()

### Heatmap

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Create mapping for variable renaming
original_columns = final_df_preparation_new.columns.to_list()
column_mapping = {col: f"Var_{i+1}" for i, col in enumerate(original_columns)}

# Apply renaming
df_renamed = df.rename(columns=column_mapping)

# Heatmap for Zone 3
zone3 = df_renamed[df_renamed[column_mapping["Soot Level Zone in the DPF"]] == 3.0]
print("✅ DataFrame size for Zone 3:", zone3.shape)
corr3 = zone3.corr()
plt.figure(figsize=(20, 14))
sns.heatmap(corr3, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, vmin=-1, vmax=1)
plt.title("Correlation Heatmap – Zone 3")
plt.tight_layout()
plt.show()

# Heatmap for Zone 4
zone4 = df_renamed[df_renamed[column_mapping["Soot Level Zone in the DPF"]] == 4.0]
print("✅ DataFrame size for Zone 4:", zone4.shape)
corr4 = zone4.corr()
plt.figure(figsize=(20, 14))
sns.heatmap(corr4, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5, vmin=-1, vmax=1)
plt.title("Correlation Heatmap – Zone 4")
plt.tight_layout()
plt.show()

# Legend table
legend_df = pd.DataFrame({
    "Original Name": original_columns,
    "Variable Code": [column_mapping[col] for col in original_columns]
})
legend_df


### Scatters

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Target variable
target = "Delta_time [h]"

# Loop over Zones (Zone 4 in this case)
for zone in [4.0]:
    print(f"📊 Plotting for Zone {int(zone)}")
    
    # Filter by zone
    plot_df = final_df_preparation_new[final_df_preparation_new["Soot Level Zone in the DPF"] == zone]

    # Drop any remaining NaNs just in case
    plot_df = plot_df.dropna()

    # Create scatter plots
    for col in plot_df.columns:
        if col == target:
            continue

        plt.figure(figsize=(6, 4))
        sns.scatterplot(
            data=plot_df,
            x=col,
            y=target,
            hue="Fuel consumed since DPF regeneration [L]",
            palette="viridis",
            alpha=0.7
        )
        plt.title(f"{target} vs {col} – Zone {int(zone)}")
        plt.grid()

        # 🧩 Optional: Increase tick density
        # x_min, x_max = plot_df[col].min(), plot_df[col].max()
        # y_min, y_max = plot_df[target].min(), plot_df[target].max()
        # plt.xticks(np.linspace(x_min, x_max, 25))
        # plt.yticks(np.linspace(y_min, y_max, 15))

        plt.xlabel(col)
        plt.ylabel(target)
        plt.tight_layout()
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.show()


## Test Section

### Zone 3

In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 📍 Zone to analyze
zone = 3

# 📁 Path where the model and scaler are saved
model_path = "./models"  # Generic relative path

# 🔧 Load model and scaler
linear_model = joblib.load(f"{model_path}/linear_model_zone{zone}.pkl")
scaler = joblib.load(f"{model_path}/scaler_linear_zone{zone}.pkl")

# 📦 Filter DataFrame by zone
df = final_df_preparation_new[
    final_df_preparation_new["Soot Level Zone in the DPF"] == zone
].copy()

# ✅ Variables to use (must match NAME and ORDER used during training)
feature_cols = [
    "fuel_intensity_index",
    "Simulated soot content in the particulate filter [kg]"
]

# 🧪 Extract and scale input features
X_new = df[feature_cols].copy()
X_new_scaled = scaler.transform(X_new)

# 🔮 Make predictions
df["Delta_time_predicted [h]"] = linear_model.predict(X_new_scaled)
print("✅ Predictions successfully added to the DataFrame.")

# 🎯 Model evaluation metrics
y_true = df["Delta_time [h]"]
y_pred = df["Delta_time_predicted [h]"]

mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"\n📊 Evaluation results – Zone {zone}:")
print(f"   MAE:  {mae:.4f}")
print(f"   RMSE: {rmse:.4f}")
print(f"   R²:   {r2:.4f}")

# 🔹 BASELINE
baseline_pred = np.full_like(y_true, fill_value=y_true.mean(), dtype=np.float64)
baseline_mae = mean_absolute_error(y_true, baseline_pred)
baseline_rmse = mean_squared_error(y_true, baseline_pred, squared=False)
baseline_r2 = r2_score(y_true, baseline_pred)

print("\n🔹 BASELINE (mean predictor):")
print(f"   MAE:  {baseline_mae:.4f}")
print(f"   RMSE: {baseline_rmse:.4f}")
print(f"   R²:   {baseline_r2:.4f}")

# 🎨 Visualizations
font_size = 16
sns.set(style="whitegrid", font_scale=1.2)

# 📈 Actual vs Predicted
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_true, y=y_pred, s=30, color="#2C3E50", edgecolor="black")
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', label="Ideal")
plt.xlabel("Actual Δtime [h]", fontsize=font_size)
plt.ylabel("Predicted Δtime [h]", fontsize=font_size)
plt.title(f"Production Test – Actual vs Predicted – Zone {zone}", fontsize=font_size + 2)
plt.tight_layout()
plt.show()

# 📉 Residuals
residuals = y_true - y_pred
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_pred, y=residuals, s=30, color="#34495E", edgecolor="black")
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Δtime [h]", fontsize=font_size)
plt.ylabel("Residuals", fontsize=font_size)
plt.title(f"Production Test – Residuals vs Predicted – Zone {zone}", fontsize=font_size + 2)
plt.tight_layout()
plt.show()


### Zone 4

In [ ]:
import pandas as pd
import joblib
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# 📍 Zone to analyze
zone = 4

# 📁 Path to load model and scaler
model_path = "./models"  # Generic path for GitHub

# 🔧 Load model and scaler
knn_model = joblib.load(f"{model_path}/knn_model_zone{zone}.pkl")
scaler = joblib.load(f"{model_path}/scaler_knn_zone{zone}.pkl")

# 📦 Filter DataFrame by zone
df = final_df_preparation_new[
    final_df_preparation_new["Soot Level Zone in the DPF"] == zone
].copy()

# ✅ Variables to use (same order and names as during training)
feature_cols = [
    "fuel_intensity_index_prev_zone",
    "Simulated soot content in the particulate filter [kg]"
]

# 🧪 Scale and predict
X_new = df[feature_cols].copy()
X_new_scaled = scaler.transform(X_new)
df["Delta_time_predicted [h]"] = knn_model.predict(X_new_scaled)

print("✅ Predictions successfully added to the DataFrame.")

# 🎯 Evaluation metrics
y_true = df["Delta_time [h]"]
y_pred = df["Delta_time_predicted [h]"]

mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)

print(f"\n📊 Evaluation results – Zone {zone}:")
print(f"   MAE:  {mae:.4f}")
print(f"   RMSE: {rmse:.4f}")
print(f"   R²:   {r2:.4f}")

# 🔹 BASELINE (mean predictor)
baseline_pred = np.full_like(y_true, fill_value=y_true.mean(), dtype=np.float64)

baseline_mae = mean_absolute_error(y_true, baseline_pred)
baseline_rmse = mean_squared_error(y_true, baseline_pred, squared=False)
baseline_r2 = r2_score(y_true, baseline_pred)

print("\n🔹 BASELINE (mean predictor):")
print(f"   MAE:  {baseline_mae:.4f}")
print(f"   RMSE: {baseline_rmse:.4f}")
print(f"   R²:   {baseline_r2:.4f}")

# 🎨 Visualization
sns.set(style="whitegrid", font_scale=1.2)
font_size = 14

# 📈 Actual vs Predicted
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_true, y=y_pred, s=30, color="#2C3E50", edgecolor="black")
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', label="Ideal")
plt.xlabel("Actual Δtime [h]", fontsize=font_size)
plt.ylabel("Predicted Δtime [h]", fontsize=font_size)
plt.title(f"Production Test – Actual vs Predicted – Zone {zone}", fontsize=font_size + 2)
plt.tight_layout()
plt.show()

# 📉 Residuals vs Predicted
residuals = y_true - y_pred
plt.figure(figsize=(6, 4))
sns.scatterplot(x=y_pred, y=residuals, s=30, color="#34495E", edgecolor="black")
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Δtime [h]", fontsize=font_size)
plt.ylabel("Residuals", fontsize=font_size)
plt.title(f"Production Test – Residuals vs Predicted – Zone {zone}", fontsize=font_size + 2)
plt.tight_layout()
plt.show()
